In [ ]:
import numpy as np
import pandas as pd
from rocketpy import Rocket, SolidMotor, Flight, Environment
from datetime import datetime
import uuid

env = Environment(
    latitude=0, 
    longitude=0, 
    elevation=0
)

env.set_date((2025, 1, 1, 12)) 

env.set_atmospheric_model(type='standard_atmosphere')

In [5]:
def generate_full_dna_dataset(iterations=1000):
    data_list = []
    
    for i in range(iterations):
        # --- A. PARÂMETROS DE MOTOR (Fabricação Própria) ---
        # Em vez de empuxo fixo, variamos a geometria do grão
        num_grains = np.random.randint(2, 8)
        grain_height = np.random.uniform(0.05, 0.15) # metros
        grain_outer_radius = np.random.uniform(0.02, 0.05) # metros
        grain_inner_radius = np.random.uniform(0.008, 0.015)
        nozzle_throat_radius = np.random.uniform(0.005, 0.012)
        
        # --- B. PARÂMETROS DE ESTRUTURA ---
        body_tube_length = np.random.uniform(0.8, 3.0)
        body_tube_diameter = grain_outer_radius * 2 + 0.02 # Folga para isolamento
        body_tube_mass = body_tube_length * 1.5 # kg/m aproximado
        
        # --- C. PARÂMETROS DE AERODINÂMICA ---
        nose_length = np.random.uniform(0.2, 0.6)
        nose_kind = np.random.choice(['conical', 'ogive', 'vonKarman'])
        
        # Aletas (DNA expandido)
        fin_n = np.random.choice([3, 4])
        fin_root = np.random.uniform(0.08, 0.30)
        fin_tip = np.random.uniform(0.02, fin_root)
        fin_span = np.random.uniform(0.05, 0.20)
        fin_sweep = np.random.uniform(0, 0.15)
        
        # --- D. SIMULAÇÃO ROCKETPY ---
        try:
            num_grains = np.random.randint(2, 8)
            grain_height = np.random.uniform(0.05, 0.15) 
            grain_sep = 0.005 # Separação de 5mm entre os grãos
            
            motor = SolidMotor(
                thrust_source=lambda t: 1000 if t < 3 else 0,
                burn_time=3.0,
                dry_mass=1.0 + (num_grains * 0.2),
                dry_inertia=(0.1, 0.1, 0.01),
                center_of_dry_mass_position=0,
                grains_center_of_mass_position=0,
                grain_density=1700,
                grain_outer_radius=grain_outer_radius,
                grain_initial_inner_radius=grain_inner_radius,
                grain_initial_height=grain_height,
                
                grain_number=num_grains,
                grain_separation=grain_sep,
                
                nozzle_radius=nozzle_throat_radius * 1.5,
                throat_radius=nozzle_throat_radius,
                interpolation_method='linear'
            )
            
            rocket = Rocket(
                radius=body_tube_diameter/2,
                mass=body_tube_mass + 2.0, # Massa seca total (tubo + eletrônica + outros)
                inertia=(10, 10, 0.1),
                center_of_mass_without_motor=body_tube_length/2,
                coordinate_system_orientation='tail_to_nose',
                power_off_drag=0.5, 
                power_on_drag=0.5
            )
            
            rocket.add_nose(length=nose_length, kind=nose_kind, position=body_tube_length)
            rocket.add_trapezoidal_fins(
                n=fin_n,
                root_chord=fin_root,
                tip_chord=fin_tip,
                span=fin_span,
                sweep_length=fin_sweep,
                position=0.1  # Distância da base (tail) para a frente (nose) ou vice-versa dependendo do seu sistema
            )
            rocket.add_motor(motor, position=0)
            
            # Simulação de Voo
            flight = Flight(rocket=rocket, environment=env, rail_length=5)
            
            # --- E. ARMAZENAMENTO ---
            data_list.append({
                'id': str(uuid.uuid4())[:8],
                # Inputs de Design
                'num_grains': num_grains, 'grain_h': grain_height, 'grain_out_r': grain_outer_radius,
                'throat_r': nozzle_throat_radius, 'tube_l': body_tube_length, 'tube_d': body_tube_diameter,
                'nose_l': nose_length, 'nose_type': nose_kind, 'fin_n': fin_n,
                'fin_root': fin_root, 'fin_tip': fin_tip, 'fin_span': fin_span, 'fin_sweep': fin_sweep,
                # Outputs de Performance
                'apogee': flight.apogee,
                'stability_initial': rocket.static_margin(0),
                'max_mach': flight.max_mach_number,
                'max_acceleration': flight.max_acceleration
            })
        except Exception as e:
            print(f"Design {i} falhou na simulação: {e}")
            continue # Ignora designs que falham fisicamente na simulação
        
    return pd.DataFrame(data_list)

In [6]:
df = generate_full_dna_dataset(500) # Comece com 500 para testar
df.to_csv('dataset/rocket_design_dataset.csv', index=False)
print("Dataset exportado com sucesso!")

capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.


KeyboardInterrupt: 